In [7]:
import logging

import matplotlib.pyplot as plt
import mlflow
import numpy as np
import pandas as pd
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from pathlib import Path

import sys
import os

# to allow importing `reproduction` from parent directory
sys.path.append(os.path.dirname(os.path.dirname(os.path.realpath("./train.ipynb"))))

from config import logger  # noqa: E402
from data_handling import CustomSpectralPipeline, load_data  # type: ignore  # noqa: E402
from reproduction import (  # noqa: E402
    major_oxides,
    masks,
    oxide_ranges,
    paper_individual_sm_rmses,
)  # noqa: E402
from utils import (  # noqa: E402
    custom_kfold_cross_validation,
    custom_train_test_split,
    filter_data_by_compositional_range,
)


mlflow.set_tracking_uri("http://localhost:5000")

dataset_loc = Path("../data/data/calib/calib_2015/1600mm/pls/")
calib_loc = Path("../data/data/calib/ccam_calibration_compositions.csv")
take_samples = None

In [4]:
logger.info("Loading data from location: %s", dataset_loc)
data = load_data(str(dataset_loc), take_samples)
logger.info("Data loaded successfully.")

Loading data from location: ../data/data/calib/calib_2015/1600mm/pls


Loading data: 100%|██████████| 414/414 [01:40<00:00,  4.13it/s]

Data loaded successfully.


In [5]:
logger.info("Initializing CustomSpectralPipeline.")
pipeline = CustomSpectralPipeline(
    masks=masks,
    composition_data_loc=calib_loc,
    major_oxides=major_oxides,
)
logger.info("Pipeline initialized. Fitting and transforming data.")
processed_data = pipeline.fit_transform(data)
logger.info("Data processing complete.")

Initializing CustomSpectralPipeline.
Pipeline initialized. Fitting and transforming data.


Transforming samples: 100%|██████████| 414/414 [00:29<00:00, 13.94it/s]


Data processing complete.


In [11]:
experiment_name = "PLS_Models_05-12-23_3"
mlflow.set_experiment(experiment_name)

2023/12/05 13:37:47 INFO mlflow.tracking.fluent: Experiment with name 'PLS_Models_05-12-23_3' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/203198903230460690', creation_time=1701779867540, experiment_id='203198903230460690', last_update_time=1701779867540, lifecycle_stage='active', name='PLS_Models_05-12-23_3', tags={}>

In [12]:
k_folds = 5
random_state = 42
n_components = 25  # paper said 20-30 TODO: grid search this
influence_plot_dir = Path("plots/")

for oxide in tqdm(major_oxides, desc="Processing oxides"):
    _oxide_ranges = oxide_ranges.get(oxide, None)
    if _oxide_ranges is None:
        logger.info("Skipping oxide: %s", oxide)
        continue

    for compositional_range in _oxide_ranges.keys():
        logger.debug(
            "Starting MLflow run for compositional range: %s, oxide: %s",
            compositional_range,
            oxide,
        )
        with mlflow.start_run(
            run_name=f"{experiment_name}_{compositional_range}_{oxide}",
        ):
            best_model = None
            best_rmse = float("inf")
            mlflow.log_param("n_components", n_components)
            mlflow.log_param("random_state", random_state)
            logger.info("Filtering data by compositional range.")
            data_filtered = filter_data_by_compositional_range(
                processed_data, compositional_range, oxide, oxide_ranges
            )

            train, test = custom_train_test_split(
                data_filtered,
                group_by="Sample Name",
                test_size=0.2,
                random_state=random_state,
            )

            logger.info("Performing custom k-fold cross-validation.")
            kf = custom_kfold_cross_validation(
                train,
                k=k_folds,
                group_by="Sample Name",
                random_state=random_state,
            )

            fold_rmse = []
            for i, (train_data, test_data) in enumerate(kf):
                logger.debug("Defining PLSRegression model.")
                pls = PLSRegression(
                    n_components=n_components
                )  # Adjust n_components as needed

                logger.debug("Extracting features and target for training.")
                X_train = train_data.drop(columns=major_oxides + ["Sample Name"])
                y_train = train_data[oxide]
                logger.debug("Extracting features and target for testing.")
                X_test = test_data.drop(columns=major_oxides + ["Sample Name"])
                y_test = test_data[oxide]

                logger.debug("Training the model.")
                pls.fit(X_train, y_train)
                logger.debug("Model training complete.")

                logger.debug("Predicting on test data.")
                y_pred = pls.predict(X_test)
                rmse = mean_squared_error(y_test, y_pred, squared=False)
                fold_rmse.append(rmse)
                logger.debug("Fold RMSE: %f", rmse)

                if rmse < best_rmse:
                    best_rmse = rmse
                    best_model = pls

            avg_rmse = sum(fold_rmse) / k_folds

            logger.debug("Logging parameters, metrics, and model to MLflow.")
            mlflow.log_params(
                {
                    "masks": masks,
                    "range": oxide_ranges[oxide][compositional_range],
                    "k_folds": k_folds,
                    "compositional_range": compositional_range,
                    "oxide": oxide,
                    "n_spectra": len(train),
                }
            )

            metrics = {
                "avg_rmse": float(avg_rmse),
                "best_rmse": float(best_rmse),
                "paper_rmse": paper_individual_sm_rmses[compositional_range][oxide],
            } | {
                f"fold_{i}_rmse": float(rmse)
                for i, rmse in enumerate(fold_rmse, start=1)
            }

            mlflow.log_metrics(metrics)
            mlflow.sklearn.log_model(
                best_model,
                "model",
                registered_model_name=f"{oxide}_{compositional_range}",
            )

            # ----- Influence Plots for Outlier Removal ----- #

            pls = PLSRegression(n_components=n_components)
            train_data = train.drop(columns=major_oxides + ["Sample Name"])
            X_train = train_data.to_numpy()
            Y_train = train[oxide].to_numpy()
            pls.fit(X_train, Y_train)

            # Calculate leverage
            t = pls.x_scores_
            leverage = np.diag(np.dot(t, np.dot(np.linalg.inv(np.dot(t.T, t)), t.T)))

            # Calculate residuals
            X_reconstructed = np.dot(t, pls.x_loadings_.T)
            residuals = X_train - X_reconstructed
            Q = np.sum(residuals**2, axis=1)

            # Plotting the influence plot
            plt.scatter(leverage, Q)
            plt.xlabel("Leverage")
            plt.ylabel("Residuals")
            plt.title("Influence Plot")
            plot_path = Path(
                influence_plot_dir
                / f"{experiment_name}/{oxide}_{compositional_range}_ip.png"
            )
            if not plot_path.parent.exists():
                plot_path.parent.mkdir(parents=True, exist_ok=True)
            plt.savefig(plot_path)
            plt.close()

            mlflow.log_artifact(str(plot_path))

            # # Identify outliers (this step is more qualitative and depends on your specific dataset)
            # outliers = identify_outliers(
            #     leverage, Q
            # )  # Implement this function based on your criteria

            # # Remove outliers and repeat the process
            # X_train = np.delete(X_train, outliers, axis=0)
            # Y_train = np.delete(Y_train, outliers, axis=0)

            logger.debug(
                "Compositional Range: %s, Oxide: %s, Average RMSE: %f",
                compositional_range,
                oxide,
                avg_rmse,
            )

Processing oxides:   0%|          | 0/8 [00:00<?, ?it/s]

Filtering data by compositional range.
Performing custom k-fold cross-validation.


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'SiO2_Full' already exists. Creating a new version of this model...
2023/12/05 13:38:15 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: SiO2_Full, version 7
Created version '7' of 

Filtering data by compositional range.
Performing custom k-fold cross-validation.


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'SiO2_Low' already exists. Creating a new version of this model...
2023/12/05 13:38:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: SiO2_Low, version 6
Created version '6' of mo

...

/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'K2O_Full' already exists. Creating a new version of this model...
2023/12/05 13:46:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: K2O_Full, version 5
Created version '5' of mo

Filtering data by compositional range.
Performing custom k-fold cross-validation.


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'K2O_Low' already exists. Creating a new version of this model...
2023/12/05 13:46:52 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: K2O_Low, version 5
Created version '5' of mode

Filtering data by compositional range.
Performing custom k-fold cross-validation.


/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'K2O_High' already exists. Creating a new version of this model...
2023/12/05 13:47:09 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: K2O_High, version 5
Created version '5' of mo

In [7]:
mlflow.end_run()